Titanic


In [26]:
import pandas as pd

df = pd.read_csv('../data/01_raw/train.csv')
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
import pandas as pd

df2 = pd.read_csv('../data/01_raw/test.csv')
df2.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Find and handle nulls

In [28]:
df_all = pd.concat([df,df2])

ss = ((100* df_all.isnull().sum() / df_all.isnull().count()).sort_values(ascending=False))
ss.index.name = 'Cat'
ss.name = 'pct'
type(ss.reset_index())  # to df
ss.reset_index()

,Cat,pct
0,Cabin,77.463713
1,Survived,31.932773
2,Age,20.091673
3,Embarked,0.152788
4,Fare,0.076394
5,Ticket,0.000000
6,Parch,0.000000
7,SibSp,0.000000
8,Sex,0.000000
9,Name,0.000000


## Strategies
- Cabin -- too many blanks and weird string data, maybe drop this.  or convert to A/B/C ... 0
- Age -- pretty important, set mean/mode as whole or by gender, pclass, etc
- Embarked -- port of embark...  set to the mode ?
- Fare is the price for each seat?  mean or mean based on class ?


In [22]:
# Age
import numpy as np

df_all = pd.concat([df,df2])
#display(df_all[df_all['Age'].isnull()])
#df_all['Age'].isnull().sum()
#df_all[['Age','Pclass']].groupby('Pclass').mean()
display(df_all[['Pclass','Sex','Age']].groupby(['Sex','Pclass']).mean())  # build groupings
#display(df_all[['Pclass','Sex','Age']].groupby(['Sex','Pclass']).transform(np.mean))  # apply to each row if null
#display(df_all['Age'])
df_all['Age'].fillna(df_all[['Pclass','Sex','Age']].groupby(['Sex','Pclass']).transform(np.mean).iloc[:,0], inplace=True)
df_all.tail()


Age
Sex    Pclass           
female 1       37.037594
       2       27.499223
       3       22.185329
male   1       41.029272
       2       30.815380
       3       25.962264

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,NaN,3,"Spector, Mr. Woolf",male,25.962264,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.000000,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.500000,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,25.962264,0,0,359309,8.0500,NaN,S
417,1309,NaN,3,"Peter, Master. Michael J",male,25.962264,1,1,2668,22.3583,NaN,C


In [25]:
# Fare avg ?
df_all[df_all['Fare'].isnull()]
df_all['Fare'].isnull().sum()
df_all[['Fare','Pclass']].groupby(['Pclass']).mean()
display(df_all[['Pclass','Sex','Fare']].groupby(['Sex','Pclass']).mean())  # build groupings
display(df_all[['Pclass','Sex','Fare']].groupby(['Sex','Pclass']).transform(np.mean))  # apply to each row if null
display(df_all['Fare'])
df_all['Fare'].fillna(df_all[['Pclass','Sex','Fare']].groupby(['Sex','Pclass']).transform(np.mean).iloc[:,0], inplace=True)
df_all.tail()
df_all['Fare'].isnull().sum()


Fare
Sex    Pclass            
female 1       109.412385
       2        23.234827
       3        15.324250
male   1        69.888385
       2        19.904946
       3        12.415462

,Fare
0,12.415462
1,109.412385
2,15.324250
3,109.412385
4,12.415462
...,...
413,12.415462
414,109.412385
415,12.415462
416,12.415462


0        7.2500
1       71.2833
2        7.9250
3       53.1000
4        8.0500
         ...   
413      8.0500
414    108.9000
415      7.2500
416      8.0500
417     22.3583
Name: Fare, Length: 1309, dtype: float64

0

# Gameplan 1.0

- Assumptions
  - Low class got stuck (as per movie)
  - Higher class folks got out first
  - Older folks may not have made it?
  - Preference to children and famlies 
  
- Likely columns:
  - Class, Sex, Age, Sibling#, Parent/Child#, Fare, Cabin 

- Maybe columns:
  - Ticket - any patern of fare/class to ticket# ? (corr to fare/class)
  - Embarked - some ports poor or worse cabins?    (corr to fare/class)

Steps
1.  Find correlations between features
2.  OneHot Encode features
3.  Run initial logistic model +/- features 



# Target Submission format

In [4]:
df = pd.read_csv('../data/01_raw/gender_submission.csv')
df.head()


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
